<table style="width:100%" border="0">
    <col width="20%">
    <col width="60%">
    <col width="20%">
    <tr></tr>
    <tr>
        <th style="text-align:left"><img src="Imagenes/UNAM_logo.jpg" width="150px"></th>
        <th style="text-align:center"><span style="font-size:220%">Espín - Boson</span></th> 
        <th style="text-align:right"><img src="Imagenes/LogoCiencias.jpg" width="170px"></th>
    </tr>
</table>

## Hamiltoniano

$$H=\frac{\Delta}{2}\sigma_{z}+\hbar\omega_{0}\left(aa^{\dagger}+\frac{1}{2}\right)+g(\sigma_{+}a+\sigma_{-}a^{\dagger})-g(\sigma_{+}a^{\dagger}+\sigma_{-}a)$$

$$H=\frac{\Delta}{2}\sigma_{z}\otimes\mathbf{I}_{N}+\mathbf{I}_{2}\otimes\hbar\omega_{0}\left(aa^{\dagger}+\frac{1}{2}\mathbf{I}_{N}\right)+g(\sigma_{+}\otimes a+\sigma_{-}\otimes a^{\dagger})-g(\sigma_{+}\otimes a^{\dagger}+\sigma_{-}\otimes a)$$

In [16]:
Paulix = [0.0 1.0;1 0]
Pauliy = [0.0 -im;im 0]
Pauliz = [1.0 0;0 -1]
Paulip = [0.0 2;0 0]
Paulim = [0.0 0;2 0]
hbar = 1.05457162853e-34

1.05457162853e-34

In [17]:
function creation(n)
    
    a = zeros(n-1)
    for i in 1:n-1
        a[i] = sqrt(i)
    end
    
    return diagm(a,-1) #reshape(diagm(a,-1),n,n)
    
end;

In [18]:
a = creation(10)'

10×10 Array{Float64,2}:
 0.0  1.0  0.0      0.0      0.0  0.0      0.0      0.0      0.0      0.0
 0.0  0.0  1.41421  0.0      0.0  0.0      0.0      0.0      0.0      0.0
 0.0  0.0  0.0      1.73205  0.0  0.0      0.0      0.0      0.0      0.0
 0.0  0.0  0.0      0.0      2.0  0.0      0.0      0.0      0.0      0.0
 0.0  0.0  0.0      0.0      0.0  2.23607  0.0      0.0      0.0      0.0
 0.0  0.0  0.0      0.0      0.0  0.0      2.44949  0.0      0.0      0.0
 0.0  0.0  0.0      0.0      0.0  0.0      0.0      2.64575  0.0      0.0
 0.0  0.0  0.0      0.0      0.0  0.0      0.0      0.0      2.82843  0.0
 0.0  0.0  0.0      0.0      0.0  0.0      0.0      0.0      0.0      3.0
 0.0  0.0  0.0      0.0      0.0  0.0      0.0      0.0      0.0      0.0

In [19]:
a'*a

10×10 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  6.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  7.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  8.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0

In [20]:
a'*a+eye(10)          #a*a'=a'a+I por propiedades de conmutación

10×10 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  6.0  0.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  7.0  0.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  8.0  0.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  10.0

In [21]:
function ket(n,m)    
    
    k = zeros(n+1)
    k[m+1] = 1
    
    return k
    
end;

In [27]:
function Hh(n,w0)
    
    a = creation(n)'
    
    hh = kron(eye(2),hbar*w0*(a*a'+0.5*eye(n)))
    
    return hh
    
end;

function Hr(n,w0)
    
    a = creation(n)'
    
    hr = kron(Paulip,a)+kron(Paulim,a')
    
    return hr
    
end;

function Ha(n,w0)
    
    a = creation(n)'
    
    ha = kron(Paulip,a')+kron(Paulim,a)
    
    return ha
    
end;

function H(n,delta,w0,g)
    
    h = (delta/2)*kron(Pauliz,eye(n))+Hh(n,w0)+g*Hr(n,w0)+g*Ha(n,w0)
    
    return h
    
end;

In [28]:
H(10,2,2,2)

20×20 Array{Float64,2}:
 1.0  0.0      0.0      0.0     0.0      …   0.0       0.0      0.0      0.0
 0.0  1.0      0.0      0.0     0.0          0.0       0.0      0.0      0.0
 0.0  0.0      1.0      0.0     0.0          0.0       0.0      0.0      0.0
 0.0  0.0      0.0      1.0     0.0          0.0       0.0      0.0      0.0
 0.0  0.0      0.0      0.0     1.0          0.0       0.0      0.0      0.0
 0.0  0.0      0.0      0.0     0.0      …   9.79796   0.0      0.0      0.0
 0.0  0.0      0.0      0.0     0.0          0.0      10.583    0.0      0.0
 0.0  0.0      0.0      0.0     0.0         10.583     0.0     11.3137   0.0
 0.0  0.0      0.0      0.0     0.0          0.0      11.3137   0.0     12.0
 0.0  0.0      0.0      0.0     0.0          0.0       0.0     12.0      0.0
 0.0  4.0      0.0      0.0     0.0      …   0.0       0.0      0.0      0.0
 4.0  0.0      5.65685  0.0     0.0          0.0       0.0      0.0      0.0
 0.0  5.65685  0.0      6.9282  0.0          0.0    